# Random Forest

## Data Preparation

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# import the prepared data set
dat = pd.read_csv('../../results/df_agg_hourly.csv')

# Check the data
dat.head()

,Standort,Date,Time,Datetime,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,Ost,Nord,...,WVv [m/s],p [hPa],Year,AnzBestWir,bezeichnung,fk_zaehler,richtung_in,richtung_out,korrekturfaktor,geometry
0,20,2023-01-01,00:00,2023-01-01 00:00,0.0,0.0,46.0,31.0,2682689,1247735,...,0.873333,972.443333,2023.0,447082.0,Militärbrücke,U15G3063864,Löwenplatz,Langstrasse,0.58,POINT (2682689 1247734.9)
1,20,2023-01-01,01:00,2023-01-01 01:00,0.0,0.0,43.0,94.0,2682689,1247735,...,1.490000,972.676667,2023.0,447082.0,Militärbrücke,U15G3063864,Löwenplatz,Langstrasse,0.58,POINT (2682689 1247734.9)
2,20,2023-01-01,02:00,2023-01-01 02:00,0.0,0.0,36.0,27.0,2682689,1247735,...,0.593333,972.620000,2023.0,447082.0,Militärbrücke,U15G3063864,Löwenplatz,Langstrasse,0.58,POINT (2682689 1247734.9)
3,20,2023-01-01,03:00,2023-01-01 03:00,0.0,0.0,22.0,27.0,2682689,1247735,...,0.670000,972.910000,2023.0,447082.0,Militärbrücke,U15G3063864,Löwenplatz,Langstrasse,0.58,POINT (2682689 1247734.9)
4,20,2023-01-01,04:00,2023-01-01 04:00,0.0,0.0,11.0,33.0,2682689,1247735,...,1.420000,972.956667,2023.0,447082.0,Militärbrücke,U15G3063864,Löwenplatz,Langstrasse,0.58,POINT (2682689 1247734.9)


In [3]:
# What are the different variables
dat.columns

Index(['Standort', 'Date', 'Time', 'Datetime', 'VELO_IN', 'VELO_OUT',
       'FUSS_IN', 'FUSS_OUT', 'Ost', 'Nord', 'Hr [%Hr]', 'RainDur [min]',
       'StrGlo [W/m2]', 'T [°C]', 'WD [°]', 'WVs [m/s]', 'WVv [m/s]',
       'p [hPa]', 'Year', 'AnzBestWir', 'bezeichnung', 'fk_zaehler',
       'richtung_in', 'richtung_out', 'korrekturfaktor', 'geometry'],
      dtype='object')

### Extract Hours and Months from the Data

In [4]:
# Extract the hours from the time column
dat['hour'] = pd.to_datetime(dat['Time']).dt.hour

# Check the data
dat['hour'].head()

/var/folders/m_/kjymw9sd0yn2wk8yjj853n_c0000gn/T/ipykernel_83370/4093744424.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dat['hour'] = pd.to_datetime(dat['Time']).dt.hour


0    0
1    1
2    2
3    3
4    4
Name: hour, dtype: int32

In [5]:
# Get the month from the Date column
dat['month'] = pd.to_datetime(dat['Date']).dt.month

# Check the data
dat['month'].head()

0    1
1    1
2    1
3    1
4    1
Name: month, dtype: int32

### Total Counts for the Bike and Foot Traffic

In [6]:
# Add up the bycicle and pedestrian counts
dat['bike_tot'] = dat['VELO_IN'] + dat['VELO_OUT']
dat['ped_tot'] = dat['FUSS_IN'] + dat['FUSS_OUT']

### NA handling

In [7]:
# Are there NAs in the data?
print(dat.isna().sum())

Standort              0
Date                  0
Time                  0
Datetime              0
VELO_IN               0
VELO_OUT              0
FUSS_IN               0
FUSS_OUT              0
Ost                   0
Nord                  0
Hr [%Hr]              0
RainDur [min]         0
StrGlo [W/m2]         0
T [°C]                0
WD [°]                0
WVs [m/s]             0
WVv [m/s]             0
p [hPa]               0
Year                  0
AnzBestWir            0
bezeichnung           0
fk_zaehler            0
richtung_in           0
richtung_out       3791
korrekturfaktor       0
geometry              0
hour                  0
month                 0
bike_tot              0
ped_tot               0
dtype: int64


In [8]:
# Let's have a look at the data types
dat.dtypes

Standort             int64
Date                object
Time                object
Datetime            object
VELO_IN            float64
VELO_OUT           float64
FUSS_IN            float64
FUSS_OUT           float64
Ost                  int64
Nord                 int64
Hr [%Hr]           float64
RainDur [min]      float64
StrGlo [W/m2]      float64
T [°C]             float64
WD [°]             float64
WVs [m/s]          float64
WVv [m/s]          float64
p [hPa]            float64
Year               float64
AnzBestWir         float64
bezeichnung         object
fk_zaehler          object
richtung_in         object
richtung_out        object
korrekturfaktor    float64
geometry            object
hour                 int32
month                int32
bike_tot           float64
ped_tot            float64
dtype: object

In [12]:
dat.columns

# Make Standort a categorical variable
dat['Standort'] = dat['Standort'].astype('category')

# Select the variables we want to use
X = dat[['Standort', 'hour', 'month', 'RainDur [min]', 'T [°C]', 'Hr [%Hr]', 'WD [°]', 'WVs [m/s]', 'p [hPa]']]
y_bike = dat['bike_tot']
y_ped = dat['ped_tot']

## Model for Bike Traffic

First we have to define the target variable and the features.

In [13]:
# Split into test and training data
X_train, X_test, y_train, y_test = train_test_split(X, y_bike, test_size=0.3, random_state=42)

# Check the shapes of the data
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(154753, 9) (66323, 9) (154753,) (66323,)


In [25]:
# Fit the model
model = RandomForestRegressor(n_estimators=300, random_state=42)
RF_bike = model.fit(X_train, y_train)

In [26]:
# Predict on test set
y_pred = RF_bike.predict(X_test)

# Calculate the root mean squared error (RMSE)
np.sqrt(mean_squared_error(y_test, y_pred))

29.79429980441965

In [27]:
np.mean(y_test)

48.78330292658655